# This notebook gives some tips and tricks I've found working with scipy sparse matrices.

In [1]:
import numpy as np
from scipy import sparse

# 1) Using ordinary dicts instead of dok_matrix for construction
Assume you want to built an interaction matrix, given a list of co-occurances. I want to show that using a dok_matrix is not the most efficient way.

In [2]:
def get_random_cooccurance_list(num_items, num_cooccurances):
    assert num_items > 0
    items = np.random.randint(num_items, size=(num_cooccurances, 2))
    return items

This method returns random pairs of items. It is meant to simulate the kind of data format you might get reading data from a long file, which might be a list of pairs.

In [3]:
num_items = 20
np.random.seed(50)
data = get_random_cooccurance_list(num_items, 10)
data

array([[16,  0],
       [11, 13],
       [ 1,  4],
       [ 6,  5],
       [ 6, 13],
       [ 5,  2],
       [ 7, 15],
       [ 4, 14],
       [ 3,  6],
       [11, 17]])

## Constructing a sparse matrix with the dok class (like scipy recommends)

In [4]:
def construct_dok_matrix(num_items, data):
    mat = sparse.dok_matrix((num_items,)*2, dtype=np.int32)
    for item1, item2 in data:
        mat[item1, item2] += 1
    return mat

Sample of how to use this method:

In [5]:
dok_mat = construct_dok_matrix(num_items, data)
dok_mat

<20x20 sparse matrix of type '<class 'numpy.int32'>'
	with 10 stored elements in Dictionary Of Keys format>

In [6]:
dok_mat.todense()

matrix([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0,

## Constructing a matrix via a default dict

In [7]:
from collections import defaultdict
def construct_default_dict_matrix(num_items, data):
    mat = defaultdict(int)
    for item1, item2 in data:
        mat[item1, item2] += 1
    
    # Convert to dok matrix at the end
    dok = sparse.dok_matrix((num_items,)*2, dtype=np.int32)
    dict.update(dok, mat)
    return dok

In [8]:
default_dict_mat = construct_default_dict_matrix(num_items, data)
default_dict_mat

<20x20 sparse matrix of type '<class 'numpy.int32'>'
	with 10 stored elements in Dictionary Of Keys format>

Make sure the two are equivalent

In [9]:
np.all(default_dict_mat.todense() == dok_mat.todense())

True

## Test performance of these two methods

In [10]:
np.random.seed(8035)
num_items_big = 100000
big_data = get_random_cooccurance_list(num_items, int(1e5))

In [11]:
%timeit construct_dok_matrix(num_items_big, big_data)

5.08 s ± 505 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
%timeit construct_default_dict_matrix(num_items_big, big_data)

248 ms ± 37.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Clearly constructing via a default dict is much faster

# 2) Column selection

In [13]:
mat = sparse.random(100, 1000000, density=1e-6, format='csr',
                   random_state=12)
mat

<100x1000000 sparse matrix of type '<class 'numpy.float64'>'
	with 99 stored elements in Compressed Sparse Row format>

## A case where converting to csc is comparable

In [14]:
cols_to_change = range(0, 1000, 2)

In [15]:
def change_via_csc(mat):
    csc_mat = mat.tocsc()
    csc_mat[:, cols_to_change] = 0
    return csc_mat.tocsr()

In [16]:
%timeit mat[:, cols_to_change] = 0

/home/austin/anaconda3/lib/python3.6/site-packages/scipy/sparse/compressed.py:742: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


4 ms ± 100 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [17]:
%timeit change_via_csc(mat)

16.1 ms ± 631 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Case where converting to csc is not better

In [18]:
cols_to_change = range(0, 10, 2)

In [19]:
%timeit mat[:, cols_to_change] = 0

499 µs ± 27.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [20]:
%timeit change_via_csc(mat)

13.6 ms ± 1.4 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


# 3) Accessing internal data structures of csr/csc
As a sample task, consider normalizing the rows of a large random sparse matrix

In [21]:
mat = sparse.random(1000, 1000, density=1e-2, format="csr")

In [22]:
mat.nnz

10000

In [23]:
def normalize_matrix_naive(mat):
    for row_idx in range(mat.shape[0]):
        row = mat.getrow(row_idx)
        mat[row_idx, :] /= row.sum() + 1e-8

In [24]:
def normalize_matrix_efficient(mat):
    for row_idx in range(mat.shape[0]):
        row_start = mat.indptr[row_idx]
        row_end = mat.indptr[row_idx+1]
        if row_start < row_end:
            total = mat.data[row_start:row_end].sum()
            mat.data[row_start:row_end] /= total

In [25]:
%timeit normalize_matrix_naive(mat)

1.22 s ± 178 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [26]:
%timeit normalize_matrix_efficient(mat)

10.3 ms ± 226 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### It is ~ 100x faster to access the internal data structures

# 4) Always use csr for adding

In [27]:
m1_coo = sparse.random(10000, 10000, 1e-3, format="coo")
m2_coo = sparse.random(10000, 10000, 1e-3, format="coo")
m1_csr = m1_coo.tocsr()
m2_csr = m2_coo.tocsr()

In [28]:
m1_coo.nnz

100000

In [29]:
%timeit m1_coo + m2_coo

20.7 ms ± 1.42 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [30]:
%timeit m1_csr + m2_csr

3.79 ms ± 126 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# 5) Converting to COO vs DOK
Scipy doesn't emphasize this very much, but the difference is extremely significant

In [31]:
mat = sparse.random(10000, 10000, density=1e-3, format='csr',
                   random_state=12)

In [32]:
mat

<10000x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 100000 stored elements in Compressed Sparse Row format>

In [33]:
%timeit mat.todok()

106 ms ± 9.98 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [34]:
%timeit mat.tocoo()

1.57 ms ± 16.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


#### Try with a much smaller matrix

In [35]:
mat = sparse.random(10000, 10000, density=1e-6, format='csr',
                   random_state=12)

In [36]:
mat

<10000x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 100 stored elements in Compressed Sparse Row format>

In [37]:
%timeit mat.todok()

293 µs ± 10.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [38]:
%timeit mat.tocoo()

124 µs ± 2.91 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


So it matters less with a small matrix, but it is still much faster to convert to coo